In [1]:
import json
import os
import gzip
from nltk.tokenize import sent_tokenize
import re
import csv
from tqdm.notebook import tqdm

from transformers import AutoTokenizer

In [2]:
input_fn = '../data/heads/head_00.jsonl.gz'
output_fn = '../out/memorization/head_train_extract.csv'
model_name = 'EleutherAI/gpt-j-6B'
max_context_len = 200

In [3]:
data_train = list(gzip.open(input_fn, 'rt'))

In [4]:
fh = open(output_fn, 'wt')
csv_writer = csv.writer(fh)

# write headers
csv_writer.writerow(["line", "sentence", "suffix", "remaining"])

32

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left')

In [6]:
for i, line in tqdm(enumerate(data_train), total=len(data_train)):
    text = json.loads(line)["text"]
    text = text.replace('\n', ' ')
    
    tokenized = tokenizer(text)
    truncated_tokens = tokenized['input_ids'][:max_context_len]
    truncated_input = tokenizer.decode(truncated_tokens)
    
    suffix_tokens = tokenized['input_ids'][max_context_len:max_context_len*2]
    suffix = tokenizer.decode(suffix_tokens)
    
    csv_writer.writerow([i, truncated_input, suffix, len(suffix_tokens) ])

  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3111 > 2048). Running this sequence through the model will result in indexing errors


In [7]:
fh.close()